# Домашнее задание к лекции «Функции потерь и оптимизация»

In [506]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

import math

In [507]:
X, y = load_iris(return_X_y=True)

filter = y != 0

X = X[filter]
y = y[filter]

X.shape, y.shape, 

((100, 4), (100,))

In [508]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

#####  Меняю значения target на 0 и 1

In [509]:
y = np.where(y == 2, 1, 0)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [510]:
def predict_proba(coefs, x):    
    return 1. / (1. + np.exp( -(x.dot(coefs[:4]) + coefs[-1]) ) )

def predict_class(coefs, x):
    probas = predict_proba(coefs, x)
    return (probas > 0.5).astype(np.float)

def bce_loss(coefs, x, y):
    probas = predict_proba(coefs, x)
    filter_ones = y == 1
    loss = -1. * (np.sum(np.log(probas[filter_ones])) + np.sum(np.log(1. - probas[~filter_ones]))) / len(y)
    return loss

def grad(coefs, x, y):
    probas = predict_proba(coefs, x)
    delta = probas - y
    modified_x = x.T * delta
    deltas = np.mean(modified_x, axis=1)
    return deltas, np.mean(delta)

##### Обучение методом градиентного спуска

In [527]:
COEFS = np.random.randn(5) # в дальнейшем везде использую такие же коэффициенты для сравнения результата

COEFS

array([-1.22853962e+00, -2.68731458e-04,  6.70880744e-01, -2.14370593e-01,
        3.98443978e-01])

In [512]:
def learn_sgd(cf, x, y, num_epochs=20, learning_rate=0.1):
    losses = []
    
    for e in range(num_epochs):
        grad_coefs, grad_bias = grad(cf, x, y)
        
        cf[:-1] = cf[:-1] - learning_rate * grad_coefs
        cf[-1] = cf[-1] - learning_rate * grad_bias
        
        loss = bce_loss(cf, x, y)
        losses.append(loss)
        
    return losses, cf

In [528]:
bce_loss(COEFS, X, y), predict_class(COEFS, X)

(2.2169465100949224,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [529]:
losses, new_coefs = learn_sgd(COEFS, X, y)

losses, new_coefs

([0.5892392571136057,
  0.5745467487760421,
  0.5622807431464154,
  0.5554970967015077,
  0.5497310419460713,
  0.5456092316960808,
  0.5419901431737402,
  0.5389525580283481,
  0.536160555218131,
  0.5335960114116806,
  0.5311515060406039,
  0.5288058885567719,
  0.5265226412806712,
  0.5242896446898996,
  0.5220938849080539,
  0.5199296053220008,
  0.5177919727222011,
  0.5156784341296673,
  0.5135870916361078,
  0.5115167985042435],
 array([-0.98079527,  0.06119298,  1.14572658,  0.05516204,  0.39196385]))

In [530]:
bce_loss(new_coefs, X, y), predict_class(new_coefs, X)

(0.5115167985042435,
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.]))

##### Обучение методом nesterov momentum

In [531]:
COEFS = np.array([-1.22853962e+00, -2.68731458e-04,  6.70880744e-01, -2.14370593e-01, 3.98443978e-01])
COEFS

array([-1.22853962e+00, -2.68731458e-04,  6.70880744e-01, -2.14370593e-01,
        3.98443978e-01])

In [532]:
bce_loss(COEFS, X, y), predict_class(COEFS, X)

(2.2169465142304428,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [533]:
def learn_nesterov(coefs, x, y, num_epochs=20, learning_rate=0.1, momentum = 0.9):
    losses = []
    
    v1 = 0 # начальная скорость изменения для первых 4-х коэффициентов
    v2 = 0 # начальная скорость изменения для последнего коэффициента
    
    for e in range(num_epochs):
        
#         Вычисляю прогнозную точку прогнозную точку и градиент для неё
        
        cf_new1 = coefs[:-1] - momentum * v1
        
        cf_new2 = coefs[-1] - momentum * v2
        
        cf_new = np.append (cf_new1, cf_new2)
        
#         Вычисляю градиент для этой точки
        
        grad_coefs, grad_bias = grad(cf_new, x, y)
        
        
#         Вычисляю скорость и направление и изменяем коэффициенты
    
        v1 = momentum * v1 + learning_rate * grad_coefs        
        coefs[:-1] = coefs[:-1] - v1
        
        v2 = momentum * v2 + learning_rate * grad_bias
        coefs[-1] = coefs[-1] - v2
        
        loss = bce_loss(coefs, x, y)
        losses.append(loss)
        
    return losses, coefs

In [534]:
losses, new_coefs = learn_sgd1(COEFS, X, y)

losses, new_coefs

[-0.90757772  0.14449245  0.94179055 -0.11536258  0.4470397 ]
[-0.88588687  0.14987877  0.99438463 -0.08541578  0.4457075 ]
[-0.92171331  0.1267549   1.01234826 -0.06311965  0.43343532]
[-0.8986586   0.12859779  1.0859893  -0.01802244  0.42881967]
[-9.99204540e-01  7.21840166e-02  1.07117465e+00  1.67022151e-04
  4.03170190e-01]
[-0.91611301  0.09829538  1.20930743  0.0738846   0.40521269]
[-1.06886143  0.01510896  1.17144568  0.09053949  0.36850736]
[-0.9874044   0.03798751  1.32497255  0.17529684  0.3678093 ]
[-1.14664901 -0.05038806  1.29689544  0.2004517   0.32785168]
[-1.08223065 -0.03724312  1.44955681  0.28930402  0.32251866]
[-1.24360321 -0.12828505  1.43008311  0.32111455  0.28058545]
[-1.1855906  -0.11951709  1.58621587  0.41427325  0.27282995]
[-1.34346265 -0.2102073   1.57627113  0.45193273  0.23027319]
[-1.29663248 -0.20758691  1.72861897  0.54581671  0.21977585]
[-1.447585   -0.29596277  1.72731265  0.587924    0.17757191]
[-1.41108324 -0.29872267  1.87323028  0.68061497 

([0.5892392579728467,
  0.549483375315287,
  0.5489392310117093,
  0.5450562124645453,
  0.5551003297257274,
  0.5948656216866726,
  0.5293356232313172,
  0.5836129461854378,
  0.49239752844644485,
  0.5443442643546722,
  0.4655821003386342,
  0.5051181655028849,
  0.43437937401506216,
  0.46298013322114,
  0.4054357958940296,
  0.4222907655984705,
  0.3774834601899183,
  0.38386950233491873,
  0.3516448341826587,
  0.34985817419412407],
 array([-1.6405943 , -0.48305229,  2.1523671 ,  0.94688496,  0.0539595 ]))

In [535]:
bce_loss(new_coefs, X, y), predict_class(new_coefs, X)

(0.34985817419412407,
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

##### Обучение методом rmsprop

In [537]:
COEFS = np.array([-1.22853962e+00, -2.68731458e-04,  6.70880744e-01, -2.14370593e-01, 3.98443978e-01])
COEFS

array([-1.22853962e+00, -2.68731458e-04,  6.70880744e-01, -2.14370593e-01,
        3.98443978e-01])

In [538]:
bce_loss(COEFS, X, y), predict_class(COEFS, X)

(2.2169465142304428,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [539]:
def learn_rmsprop(coefs, x, y, num_epochs=20, learning_rate=0.1, momentum = 0.9):
    losses = []
    
    E1 = 0 # бегущее среднее для первых 4-х коэффициентов
    E2 = 0 # бегущее среднее для последнего коэффициента
    
    for e in range(num_epochs):
        
#         Считаю гнрадиенты
        grad_coefs, grad_bias = grad(coefs, x, y)
    
#         Считаю новое среднее для каждого из 4-х коэффициентов, обновляю коэффициенты
        E1_new = []        

        for j, coef in enumerate(grad_coefs):            
            e_prev = E1 if e == 0 else E1[j]    

            e_new = momentum * e_prev + (1 - momentum) * (grad_coefs[j]**2)
            
            E1_new.append(e_new)
            
            coefs[j] = coefs[j] - learning_rate / math.sqrt(e_new + 10**-6) * grad_coefs[j]

#         Обновляю E1
        E1 = E1_new
    
#         Считаю новое среднее для последнего коэффициента, обновляю коэффициент
    
        E2 = momentum * E2 + (1 - momentum) * (grad_bias**2)
        coefs[-1] = coefs[-1] - learning_rate / math.sqrt(E2 + 10**-6) * grad_bias  
        
        loss = bce_loss(coefs, x, y)
        losses.append(loss)
        
    return losses, coefs

In [540]:
losses, new_coefs = learn_rmsprop(COEFS, X, y)

losses, new_coefs

([0.6196766282004638,
  0.647687603925824,
  0.6888399853672199,
  0.6627117779357813,
  0.6624969848984767,
  0.617280893914502,
  0.6109145555444904,
  0.5855900592971486,
  0.5839040882305945,
  0.569913118691748,
  0.5729218239967893,
  0.5640860920434432,
  0.5709096021771473,
  0.562939624115715,
  0.5714580059685381,
  0.5606645991322922,
  0.5681085229621021,
  0.5537428543331082,
  0.5592281583948978,
  0.5436728962245178],
 array([-1.11333712,  0.01612747,  1.06640918,  0.45751202,  0.27974223]))

In [541]:
bce_loss(new_coefs, X, y), predict_class(new_coefs, X)

(0.5436728962245178,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.]))

### Выводы


На этом дата сете лучшие результаты показывает nesterov momentum, но в целом результат зависит от попадания в первую точку и кол-ва итераций. Также результаты лучше, если обучать на первых двух группах ирисов (убирать y == 2, а не y == 0), там зависимость прослеживается лучше.

<i>Хардкод начальных коэффициентов в двух последних методах для того, чтобы можно было наглядно сравнить результаты </i>